# Inroduction
Many ML problems involve thousands or even milions of features which maje training extremely slow and maybe harder to find a good solution --> curse of dimensionality

There are two main approaches: 
1. projection 
2. manifold learning

## Heads up
Reducing dimensionality does cause some information loss (compressing an image to JPEG can degrade its quality). It speeds up training but performs slightly worse (in rare cases you may filter noise and unnecessary data), it also makes your pipeline a bit more complex --> first try the original data

## Applications
1. speed up
2. data visualization --> detecting patterns and clusters


# The Curse of Dimensionality
If you pick a random point in a unit square, it will have only about 0.4% chance of being located less than 0.001 from a border (it is very unlikely that a random point will be "extreme" along any dimenstion. In a 10,000 dimensional unit hypercube, this probablity is greater than 99.999999%

![](img/photo_2023-04-03_17-47-43.jpg)

If you pick two points randomly in a unit square the distance between them will be on avrage roghly 0.52. If you pick two random points in a 3D unit cube, the average distance will be roughly 0.66. What about two points picked randomly in a 1,000,000-dimensional unit hypercube? The average distance, will be about 408.25. 

This is counterintuitive: how can two points be so far apart when they both lie within the same unit hypercube? Well, there’s just plenty of space in high dimensions. As a result, high-dimensional datasets are at risk of being very sparse: most training instances are likely to be far away from each other. In short, the more dimensions the training set has, the greater the risk of overfitting it.

In theory, one solution to the curse of dimensionality could be to increase the size of the training set to reach a sufficient density of training instances. In practice, the number of training instances required to reach a given density grows exponentially with the number of dimensions. With just 100 features—significantly fewer than in the MNIST problem all ranging from 0 to 1, you would need more training instances than atoms in the observable universe in order for training instances to be within 0.1 of each other on average, assuming they were spread out uniformly across all dimensions.

# Projection
Training instances are not spread out uniformly across all dimenstions
1. Many features are almost constant
2. features are highly correlated
Like MNIST dataset.

Lie withing or close to a much lower dimensional subspaces
![](img/photo_2023-04-03_18-16-02.jpg)

# Manifold Learning
![](img/photo_2023-04-03_18-27-23.jpg)

![](img/photo_2023-04-03_18-27-27.jpg)

Squashing by projecting onto a plane (left) versus unrolling the Swiss roll (right)

## Heads up

![](img/photo_2023-04-03_18-44-40.jpg)
The dicision boundary may not always be simpler with lower dimentions

# Principal component analysis (PCA)

## Preserving the Variance
![](img/photo_2023-04-03_19-09-02.jpg)
select the axis that preserves the maximum amount of variance, as it will most likely lose less information (+the axis that minimizes the mean squared distance between the original dataset and its projection onto that axis+)

## Principal Components
1. Zero-centered unit vector in the direction of the PC
2. They are orthogonal to each other
+Since two opposing unit vectors lie on the same axis, the direction of the unit vectors returned by PCA is not stable+

## How to find them?
Standard matrix factorization technique called singular value decomposition (SVD) decomposes the training set matrix $X$ into the matrix multiplication of three matrices $U \sum V^T$ where $V$ contains the unit vectors

In [29]:
import numpy as np

# Define the number of data points
n = 10

# Generate random data for the x, y, and z coordinates
x = np.random.rand(n)
y = np.random.rand(n)
z = np.random.rand(n)

# Combine the x, y, and z coordinates into a single dataset
data = np.column_stack((x, y, z))

data_centered = data - data.mean(axis=0)
U, s, Vt = np.linalg.svd(data_centered)
c1 = Vt[0]
c2 = Vt[1]

print("c1: ", c1)
print("c2: ", c2)

# Heads up
# PCA assumes that the dataset is centered around the origin, Scikit-Learn's PCA classes take care of centering

c1:  [-0.7769242   0.16085852  0.60869806]
c2:  [ 0.22765677 -0.82961563  0.509814  ]


In [41]:
print(s)

[1.33872797 0.80449451 0.65101911]


## Projecting Down to d  Dimensions 
Project onto the first d principal components.

In [30]:
W2 = Vt[:2].T
W2D = data_centered @ W2
W2D

array([[-0.52198892, -0.06160485],
       [-0.02403791,  0.22465357],
       [-0.64316296, -0.21995502],
       [-0.35694905,  0.30980858],
       [ 0.36558658,  0.3628524 ],
       [-0.36707219,  0.1160826 ],
       [ 0.37629299, -0.2077425 ],
       [ 0.11914735, -0.50292153],
       [ 0.51310426, -0.07032475],
       [ 0.53907985,  0.04915149]])

In [31]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(data)
X2D

array([[ 0.52198892,  0.06160485],
       [ 0.02403791, -0.22465357],
       [ 0.64316296,  0.21995502],
       [ 0.35694905, -0.30980858],
       [-0.36558658, -0.3628524 ],
       [ 0.36707219, -0.1160826 ],
       [-0.37629299,  0.2077425 ],
       [-0.11914735,  0.50292153],
       [-0.51310426,  0.07032475],
       [-0.53907985, -0.04915149]])

In [32]:
print(pca.components_)

[[ 0.7769242  -0.16085852 -0.60869806]
 [-0.22765677  0.82961563 -0.509814  ]]


In [33]:
print(pca.explained_variance_ratio_)

[0.62593388 0.22604242]


## Choosing the right number of dimensions
Choose the number of dimensioins that add up to a sufficiently large portion of the variance--say, 95%

In [8]:
# from sklearn.datasets import fetch_openml

# mnist = fetch_openml('mnist_784', as_frame=False)

# X_train, y_train = mnist.data[:60_000], mnist.target[:60_000]
# X_test, y_test = mnist.data[60_000:], mnist.target[60_000:]

# pca = PCA()
# pca.fit(X_train)

# cumsum = np.cumsum(pca.explained_variance_ratio_)
# d = np.argmax(cumsum >= 0.95) + 1 # d equals 154
# d

In [34]:
pca = PCA(n_components=0.95)
data_reduced = pca.fit_transform(data)
data_reduced

array([[ 0.52198892,  0.06160485,  0.22322649],
       [ 0.02403791, -0.22465357, -0.05645224],
       [ 0.64316296,  0.21995502, -0.05357744],
       [ 0.35694905, -0.30980858,  0.22095859],
       [-0.36558658, -0.3628524 , -0.27990281],
       [ 0.36707219, -0.1160826 , -0.24994704],
       [-0.37629299,  0.2077425 ,  0.29645003],
       [-0.11914735,  0.50292153, -0.27262856],
       [-0.51310426,  0.07032475,  0.06027031],
       [-0.53907985, -0.04915149,  0.11160267]])

In [35]:
pca.n_components_

3

![](img/photo_2023-04-03_20-49-18.jpg)

In [ ]:
# Randomized Search CV

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.pipeline import make_pipeline

# clf = make_pipeline(PCA(random_state=42),
#  RandomForestClassifier(random_state=42))
# param_distrib = {
#  "pca__n_components": np.arange(10, 80),
#  "randomforestclassifier__n_estimators": np.arange(50, 500)
# }
# rnd_search = RandomizedSearchCV(clf, param_distrib, n_iter=10, cv=3,
#  random_state=42)
# rnd_search.fit(X_train[:1000], y_train[:1000])
# print(rnd_search.best_params_)

# PCA for Compression
1. apply PCA to the MNIST
2. 154 features instead of 784
3. less than 20% of its original size, we only lost 5% of its variance
4. possible to decompress (inverse transformation), this won't give you back the original data
5. likely be close to the original data
6. mean squared distance between original data and the reconstructed data is called the reconstruction error

$X_{recovered} = X_{d-proj} * W_d^T$

$W_d$ is orthogonal so $W_d^{-1} = W_d^T$

In [36]:
data_recovered = pca.inverse_transform(data_reduced)
data_recovered

array([[0.92313538, 0.63253196, 0.33211065],
       [0.4372643 , 0.32561384, 0.6111265 ],
       [0.81874687, 0.59641357, 0.00934627],
       [0.87813544, 0.34973709, 0.62054314],
       [0.03485351, 0.15416603, 0.78290402],
       [0.56547886, 0.25705189, 0.22934031],
       [0.23495094, 0.93741611, 0.84890472],
       [0.03348875, 0.83667301, 0.19593551],
       [0.0213071 , 0.71914335, 0.85865872],
       [0.05845736, 0.65164781, 0.96658705]])

In [37]:
data

array([[0.92313538, 0.63253196, 0.33211065],
       [0.4372643 , 0.32561384, 0.6111265 ],
       [0.81874687, 0.59641357, 0.00934627],
       [0.87813544, 0.34973709, 0.62054314],
       [0.03485351, 0.15416603, 0.78290402],
       [0.56547886, 0.25705189, 0.22934031],
       [0.23495094, 0.93741611, 0.84890472],
       [0.03348875, 0.83667301, 0.19593551],
       [0.0213071 , 0.71914335, 0.85865872],
       [0.05845736, 0.65164781, 0.96658705]])

![](img/photo_2023-04-03_21-03-34.jpg) 
MNIST compression that preserves 95% of the variance

In [38]:
# Randomized PCA
# finds an approximation of the first d principal components, it's dramatically faster in some cases

rnd_pca = PCA(n_components=2, svd_solver="randomized", random_state=42)
X_reduced = rnd_pca.fit_transform(data)
X_reduced

array([[ 0.52198892,  0.06160485],
       [ 0.02403791, -0.22465357],
       [ 0.64316296,  0.21995502],
       [ 0.35694905, -0.30980858],
       [-0.36558658, -0.3628524 ],
       [ 0.36707219, -0.1160826 ],
       [-0.37629299,  0.2077425 ],
       [-0.11914735,  0.50292153],
       [-0.51310426,  0.07032475],
       [-0.53907985, -0.04915149]])

In [39]:
X2D

array([[ 0.52198892,  0.06160485],
       [ 0.02403791, -0.22465357],
       [ 0.64316296,  0.21995502],
       [ 0.35694905, -0.30980858],
       [-0.36558658, -0.3628524 ],
       [ 0.36707219, -0.1160826 ],
       [-0.37629299,  0.2077425 ],
       [-0.11914735,  0.50292153],
       [-0.51310426,  0.07032475],
       [-0.53907985, -0.04915149]])

## Incremental PCA
PCA requires the whole training set to fit in memory. Incremental PCA (IPCA) uses mini-batches and is useful for large training sets and PCA online (i.e, on the fly, as new insatnces arrive)

In [ ]:
# must call the partial_fit() method with each mini-batch,
from sklearn.decomposition import IncrementalPCA
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)
X_reduced = inc_pca.transform(X_train)

Alternatively, you can use NumPy’s memmap class, manipulates a
large array stored in a binary file on disk as if it were entirely in memory; the class
loads only the data it needs in memory, when it needs it.

In [ ]:
filename = "my_mnist.mmap"
X_mmap = np.memmap(filename, dtype='float32', mode='write', shape=X_train.shape)
X_mmap[:] = X_train # could be a loop instead, saving the data chunk by chunk
X_mmap.flush()

We can load the memmap file and use it like a regular NumPy array. Since this algorithm uses only
a small part of the array at any given time, memory usage remains under control.
This makes it possible to call the usual fit() method instead of partial_fit()

In [ ]:
X_mmap = np.memmap(filename, dtype="float32", mode="readonly").reshape(-1, 784)
batch_size = X_mmap.shape[0] // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mmap)

Only the raw binary data is saved to disk --> specify the data type and shape of the array, np.memmap() returns a 1D array.